# Validation of Continous Loss Function from R File python conversion.qmd

In [1]:
import torch
import numpy as np
import torch
import torch
import torch.nn.functional as F

torch.set_printoptions(precision=8)

Data

In [2]:
#train$df_orig
# tf.Tensor(
# [[ 0.38709584 -0.18872575  3.9523134 ]
#  [ 0.7616431  -0.11664478 -1.4364368 ]], shape=(2, 3), dtype=float32)

# min max for tensors q5 nd q95
# tf.Tensor([ 0.4058232  -0.18512172 -1.1669992 ], shape=(3), dtype=float32)
# tf.Tensor([ 0.74291575 -0.12024883  3.6828759 ], shape=(3), dtype=float32)

# h_params
# tf.Tensor(
# [[[ 0.          0.         -0.05309175 -0.02268984 -0.00073227
#     0.14473043 -0.19736235 -0.01326796  0.01716295  0.04003201
#    -0.03761906  0.02019005  0.02103117  0.01475964  0.02561173
#     0.04465405  0.07558934 -0.03455871  0.03127078 -0.06442269
#    -0.03358709 -0.0046008 ]

#   [ 0.          0.03179859 -0.03748546  0.03065695  0.01450718
#     0.05038024  0.08786372 -0.01381071  0.08681878  0.03438636
#    -0.11747339 -0.07255729 -0.02515779 -0.06152298  0.01535351
#     0.01071469  0.00725646 -0.02823549 -0.01658672 -0.031664
#     0.06521034 -0.05619556]

#   [ 0.06369114  0.          0.07416828 -0.02284147  0.02354072
#     0.06744125 -0.09243326  0.03462008  0.08162504  0.09570658
#    -0.06736961 -0.04148068  0.03080387  0.07890061 -0.0014052
#     0.0158043   0.06599072  0.01494174 -0.05903206 -0.00370814
#     0.00470866  0.04583171]]

#  [[ 0.          0.         -0.05309175 -0.02268984 -0.00073227
#     0.14473043 -0.19736235 -0.01326796  0.01716295  0.04003201
#    -0.03761906  0.02019005  0.02103117  0.01475964  0.02561173
#     0.04465405  0.07558934 -0.03455871  0.03127078 -0.06442269
#    -0.03358709 -0.0046008 ]

#   [ 0.          0.06256635 -0.03748546  0.03065695  0.01450718
#     0.05038024  0.08786372 -0.01381071  0.08681878  0.03438636
#    -0.11747339 -0.07255729 -0.02515779 -0.06152298  0.01535351
#     0.01071469  0.00725646 -0.02823549 -0.01658672 -0.031664
#     0.06521034 -0.05619556]

#   [ 0.06369113  0.          0.07416828 -0.02284142  0.02354072
#     0.06744127 -0.09243318  0.0346201   0.08162504  0.09570658
#    -0.06736961 -0.04148067  0.03080386  0.07890059 -0.0014052
#     0.01580431  0.06599071  0.01494177 -0.05903205 -0.00370815
#     0.00470866  0.04583171]]], shape=(2, 3, 22), dtype=float32)

## R -> to_Theta, Python transform_intercepts_continpus

In [3]:

def transform_intercepts_continous(theta_tilde:torch.Tensor) -> torch.Tensor:
    
    """
    Transforms the unordered theta_tilde to ordered theta values for the bernstein polynomial
    E.G: 
    theta_1 = theta_tilde_1
    theta_2 = theta_tilde_1 + exp(theta_tilde_2)
    ..
    :param theta_tilde: The unordered theta_tilde values
    :return: The ordered theta values
    """

    # Compute the shift based on the last dimension size
    last_dim_size = theta_tilde.shape[-1]
    shift = torch.log(torch.tensor(2.0)) * last_dim_size / 2

    # Get the width values by applying softplus from the second position onward
    widths = torch.nn.functional.softplus(theta_tilde[..., 1:])

    # Concatenate the first value (raw) with the softplus-transformed widths
    widths = torch.cat([theta_tilde[..., [0]], widths], dim=-1)

    # Return the cumulative sum minus the shift
    return torch.cumsum(widths, dim=-1) - shift


In [4]:
# theta tilde


# tf.Tensor(
# [[[-0.05309175 -0.02268984 -0.00073227  0.14473043 -0.19736235
#    -0.01326796  0.01716295  0.04003201 -0.03761906  0.02019005
#     0.02103117  0.01475964  0.02561173  0.04465405  0.07558934
#    -0.03455871  0.03127078 -0.06442269 -0.03358709 -0.0046008 ]
#   [-0.03748546  0.03065695  0.01450718  0.05038024  0.08786372
#    -0.01381071  0.08681878  0.03438636 -0.11747339 -0.07255729
#    -0.02515779 -0.06152298  0.01535351  0.01071469  0.00725646
#    -0.02823549 -0.01658672 -0.031664    0.06521034 -0.05619556]
#   [ 0.07416828 -0.02284147  0.02354072  0.06744125 -0.09243326
#     0.03462008  0.08162504  0.09570658 -0.06736961 -0.04148068
#     0.03080387  0.07890061 -0.0014052   0.0158043   0.06599072
#     0.01494174 -0.05903206 -0.00370814  0.00470866  0.04583171]]

#  [[-0.05309175 -0.02268984 -0.00073227  0.14473043 -0.19736235
#    -0.01326796  0.01716295  0.04003201 -0.03761906  0.02019005
#     0.02103117  0.01475964  0.02561173  0.04465405  0.07558934
#    -0.03455871  0.03127078 -0.06442269 -0.03358709 -0.0046008 ]
#   [-0.03748546  0.03065695  0.01450718  0.05038024  0.08786372
#    -0.01381071  0.08681878  0.03438636 -0.11747339 -0.07255729
#    -0.02515779 -0.06152298  0.01535351  0.01071469  0.00725646
#    -0.02823549 -0.01658672 -0.031664    0.06521034 -0.05619556]
#   [ 0.07416828 -0.02284142  0.02354072  0.06744127 -0.09243318
#     0.0346201   0.08162504  0.09570658 -0.06736961 -0.04148067
#     0.03080386  0.07890059 -0.0014052   0.01580431  0.06599071
#     0.01494177 -0.05903205 -0.00370815  0.00470866  0.04583171]]], shape=(2, 3, 20), dtype=float32)


# tf.Tensor(
# [[[-6.9845634e+00 -6.3026972e+00 -5.6099157e+00 -4.8417873e+00
#    -4.2424603e+00 -3.5559251e+00 -2.8541598e+00 -2.1407962e+00
#    -1.4662814e+00 -7.6298809e-01 -5.9269905e-02  6.4128447e-01
#     1.3473196e+00  2.0630426e+00  2.7946987e+00  3.4707155e+00
#     4.1796207e+00  4.8410749e+00  5.5175695e+00  6.2084188e+00]
#   [-6.9689574e+00 -6.2603641e+00 -5.5599370e+00 -4.8412828e+00
#    -4.1032391e+00 -3.4169731e+00 -2.6794748e+00 -1.9689865e+00
#    -1.3328519e+00 -6.7532539e-01  5.3219795e-03  6.6818047e-01
#     1.3690338e+00  2.0675526e+00  2.7643347e+00  3.4434633e+00
#     4.1283512e+00  4.8057919e+00  5.5320759e+00  6.1975203e+00]
#   [-6.8573036e+00 -6.1755118e+00 -5.4705248e+00 -4.7430887e+00
#    -4.0950909e+00 -3.3844838e+00 -2.6496916e+00 -1.9075465e+00
#    -1.2475171e+00 -5.7489538e-01  1.3377237e-01  8.6714792e-01
#     1.5595932e+00  2.2606735e+00  2.9873600e+00  3.6880064e+00
#     4.3520727e+00  5.0433674e+00  5.7388716e+00  6.4551973e+00]]

#    [-6.9845634e+00 -6.3026972e+00 -5.6099157e+00 -4.8417873e+00
#    -4.2424603e+00 -3.5559251e+00 -2.8541598e+00 -2.1407962e+00
#    -1.4662814e+00 -7.6298809e-01 -5.9269905e-02  6.4128447e-01
#     1.3473196e+00  2.0630426e+00  2.7946987e+00  3.4707155e+00
#     4.1796207e+00  4.8410749e+00  5.5175695e+00  6.2084188e+00]
#   [-6.9689574e+00 -6.2603641e+00 -5.5599370e+00 -4.8412828e+00
#    -4.1032391e+00 -3.4169731e+00 -2.6794748e+00 -1.9689865e+00
#    -1.3328519e+00 -6.7532539e-01  5.3219795e-03  6.6818047e-01
#     1.3690338e+00  2.0675526e+00  2.7643347e+00  3.4434633e+00
#     4.1283512e+00  4.8057919e+00  5.5320759e+00  6.1975203e+00]
#   [-6.8573036e+00 -6.1755118e+00 -5.4705248e+00 -4.7430887e+00
#    -4.0950909e+00 -3.3844838e+00 -2.6496916e+00 -1.9075465e+00
#    -1.2475171e+00 -5.7489538e-01  1.3377237e-01  8.6714792e-01
#     1.5595932e+00  2.2606735e+00  2.9873600e+00  3.6880064e+00
#     4.3520727e+00  5.0433674e+00  5.7388716e+00  6.4551973e+00]]], shape=(2, 3, 20), dtype=float32)



In [43]:
# thetas from Model 

theta_tilde_r = torch.tensor([
    [-0.05309175, -0.02268984, -0.00073227,  0.14473043, -0.19736235,
     -0.01326796,  0.01716295,  0.04003201, -0.03761906,  0.02019005,
      0.02103117,  0.01475964,  0.02561173,  0.04465405,  0.07558934,
     -0.03455871,  0.03127078, -0.06442269, -0.03358709, -0.0046008],

    [-0.05309175, -0.02268984, -0.00073227,  0.14473043, -0.19736235,
     -0.01326796,  0.01716295,  0.04003201, -0.03761906,  0.02019005,
      0.02103117,  0.01475964,  0.02561173,  0.04465405,  0.07558934,
     -0.03455871,  0.03127078, -0.06442269, -0.03358709, -0.0046008]
])

theta_r = torch.tensor([
    [-6.9845634e+00, -6.3026972e+00, -5.6099157e+00, -4.8417873e+00,
     -4.2424603e+00, -3.5559251e+00, -2.8541598e+00, -2.1407962e+00,
     -1.4662814e+00, -7.6298809e-01, -5.9269905e-02,  6.4128447e-01,
      1.3473196e+00,  2.0630426e+00,  2.7946987e+00,  3.4707155e+00,
      4.1796207e+00,  4.8410749e+00,  5.5175695e+00,  6.2084188e+00],

    [-6.9845634e+00, -6.3026972e+00, -5.6099157e+00, -4.8417873e+00,
     -4.2424603e+00, -3.5559251e+00, -2.8541598e+00, -2.1407962e+00,
     -1.4662814e+00, -7.6298809e-01, -5.9269905e-02,  6.4128447e-01,
      1.3473196e+00,  2.0630426e+00,  2.7946987e+00,  3.4707155e+00,
      4.1796207e+00,  4.8410749e+00,  5.5175695e+00,  6.2084188e+00]
])

theta_tilde_r   # same data see the to samples are the same


tensor([[-0.05309175, -0.02268984, -0.00073227,  0.14473043, -0.19736235,
         -0.01326796,  0.01716295,  0.04003201, -0.03761906,  0.02019005,
          0.02103117,  0.01475964,  0.02561173,  0.04465405,  0.07558934,
         -0.03455871,  0.03127078, -0.06442269, -0.03358709, -0.00460080],
        [-0.05309175, -0.02268984, -0.00073227,  0.14473043, -0.19736235,
         -0.01326796,  0.01716295,  0.04003201, -0.03761906,  0.02019005,
          0.02103117,  0.01475964,  0.02561173,  0.04465405,  0.07558934,
         -0.03455871,  0.03127078, -0.06442269, -0.03358709, -0.00460080]])

In [50]:
thetas_python=transform_intercepts_continous(theta_tilde_r)
thetas_python

tensor([[-6.98456335, -6.30269718, -5.60991573, -4.84178734, -4.24246025,
         -3.55592513, -2.85415936, -2.14079618, -1.46628141, -0.76298857,
         -0.05927038,  0.64128399,  1.34731865,  2.06304264,  2.79469872,
          3.47071552,  4.17961979,  4.84107494,  5.51756954,  6.20841885],
        [-6.98456335, -6.30269718, -5.60991573, -4.84178734, -4.24246025,
         -3.55592513, -2.85415936, -2.14079618, -1.46628141, -0.76298857,
         -0.05927038,  0.64128399,  1.34731865,  2.06304264,  2.79469872,
          3.47071552,  4.17961979,  4.84107494,  5.51756954,  6.20841885]])

In [51]:
theta_r-thetas_python

tensor([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -4.76837158e-07,  0.00000000e+00,
          0.00000000e+00,  4.76837158e-07,  4.76837158e-07,  4.76837158e-07,
          9.53674316e-07,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          9.53674316e-07,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -4.76837158e-07,  0.00000000e+00,
          0.00000000e+00,  4.76837158e-07,  4.76837158e-07,  4.76837158e-07,
          9.53674316e-07,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          9.53674316e-07,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

- works Same output as R CODE 

## h_dag_extra calls hdag  and hdag dash which call bernstein poly

In [46]:
L_START=0.0001
R_START=1-L_START

### Bernstein basis 

In [47]:
def bernstein_basis(tensor, M):
    """
    Compute the Bernstein basis polynomials for a given input tensor.
    Args:
        tensor (torch.Tensor): Input tensor of shape (n_samples).
        M (int): Degree of the Bernstein polynomial.
    Returns:
        torch.Tensor: Tensor of shape (B, Nodes, M+1) with the Bernstein basis.
    """
    tensor = torch.as_tensor(tensor)
    dtype = tensor.dtype
    M = torch.tensor(M, dtype=dtype, device=tensor.device)

    # Expand dims to allow broadcasting
    tensor_expanded = tensor.unsqueeze(-1)  # shape (B, Nodes, 1)

    # Clip values to avoid log(0)
    eps = torch.finfo(dtype).eps
    tensor_expanded = torch.clamp(tensor_expanded, min=eps, max=1 - eps)

    k_values = torch.arange(M + 1, dtype=dtype, device=tensor.device)  # shape (M+1,)
    
    # Log binomial coefficient: log(M choose k)
    log_binomial_coeff = (
        torch.lgamma(M + 1) 
        - torch.lgamma(k_values + 1) 
        - torch.lgamma(M - k_values + 1)
    )

    # Log powers
    log_powers = (
        k_values * torch.log(tensor_expanded)
        + (M - k_values) * torch.log(1 - tensor_expanded)
    )

    # Bernstein basis in log space
    log_bernstein = log_binomial_coeff + log_powers  # shape (B, Nodes, M+1)

    return torch.exp(log_bernstein)

def test_bernstein_basis_1d():
    samples = torch.tensor([0.2, 0.4, 0.4], dtype=torch.float32)
    M = 4
    output = bernstein_basis(samples, M)

    # Should return shape (N, M+1)
    assert output.shape == (len(samples), M + 1), f"Expected shape {(len(samples), M + 1)}, got {output.shape}"

    # All values should be >= 0
    assert torch.all(output >= 0), "All Bernstein basis values should be non-negative"

    # Each row should sum to ~1
    row_sums = output.sum(dim=1)
    assert torch.allclose(row_sums, torch.ones_like(row_sums), atol=1e-6), f"Sums were not 1: {row_sums}"

test_bernstein_basis_1d()

def h_dag(targets: torch.Tensor, thetas: torch.Tensor) -> torch.Tensor:
    """
    Args:
        targets: shape (n,)
        thetas: shape (n, b)
    Returns:
        Tensor: shape (n,)
    """
    _, b = thetas.shape
    B = bernstein_basis(targets, b - 1)  # shape (n, b)
    return torch.mean(B * thetas, dim=1)

def h_dag_dash(targets: torch.Tensor, thetas: torch.Tensor) -> torch.Tensor:
    """
    Args:
        targets: shape (n,)
        thetas: shape (n, b)
    Returns:
        Tensor: shape (n,)
    """
    _, b = thetas.shape
    dtheta = thetas[:, 1:] - thetas[:, :-1]         # shape (n, b-1)
    B_dash = bernstein_basis(targets, b - 2)        # shape (n, b-1)
    return torch.sum(B_dash * dtheta, dim=1)

###  h_extarpolated

In [52]:
#train$df_orig
# tf.Tensor(
# [[ 0.38709584 -0.18872575  3.9523134 ]
#  [ 0.7616431  -0.11664478 -1.4364368 ]], shape=(2, 3), dtype=float32)

# min max for tensors q5 nd q95
# tf.Tensor([ 0.4058232  -0.18512172 -1.1669992 ], shape=(3), dtype=float32)
# tf.Tensor([ 0.74291575 -0.12024883  3.6828759 ], shape=(3), dtype=float32)


targets=torch.tensor([ 0.38709584, 0.7616431 ])
min=0.4058232
max=0.74291575

In [53]:
def h_extrapolated(thetas: torch.Tensor, targets: torch.Tensor, k_min: float, k_max: float) -> torch.Tensor:
    """
    Args:
        thetas: shape (n, b)
        targets: shape (n,)
        k_min: float, lower bound of scaling (not tracked in graph)
        k_max: float, upper bound of scaling (not tracked in graph)
    Returns:
        Tensor of shape (n,)
    """
    # Constants (not part of the graph)
    L_START = 0.0
    R_START = 1.0

    # Detach constants from graph
    L_tensor = torch.tensor(L_START, dtype=targets.dtype, device=targets.device)
    R_tensor = torch.tensor(R_START, dtype=targets.dtype, device=targets.device)

    # Scale targets
    t_i = (targets - k_min) / (k_max - k_min)  # shape (n,)
    t_i_exp = t_i.unsqueeze(-1)  # shape (n, 1)

    # Extrapolation at left (t_i < 0)
    b0 = h_dag(L_tensor.expand_as(targets), thetas).unsqueeze(-1)     # (n, 1)
    slope0 = h_dag_dash(L_tensor.expand_as(targets), thetas).unsqueeze(-1)  # (n, 1)
    h_left = slope0 * (t_i_exp - L_tensor) + b0

    # Start with placeholder
    h = h_left.clone()

    # Mask for left extrapolation
    mask0 = t_i_exp < L_tensor
    h = torch.where(mask0, h_left, t_i_exp)  # placeholder fill

    # Extrapolation at right (t_i > 1)
    b1 = h_dag(R_tensor.expand_as(targets), thetas).unsqueeze(-1)
    slope1 = h_dag_dash(R_tensor.expand_as(targets), thetas).unsqueeze(-1)
    h_right = slope1 * (t_i_exp - R_tensor) + b1

    mask1 = t_i_exp > R_tensor
    h = torch.where(mask1, h_right, h)

    # In-domain: t_i ∈ [0,1]
    mask_mid = (t_i_exp >= L_tensor) & (t_i_exp <= R_tensor)
    h_center = h_dag(t_i, thetas).unsqueeze(-1)
    h = torch.where(mask_mid, h_center, h)

    return h.squeeze(-1)



#output h_dag extra
# h_dag_extra(t_i[,cont_dims, drop=FALSE],
#                     theta[,cont_dims,
#                     1:len_theta,drop=FALSE],
#                     k_min[cont_dims],
#                     k_max[cont_dims])

# tf.Tensor(
# [[-0.38711396 -0.38781664  0.36255726]
#  [ 0.34880346  0.34685454 -0.38074815]], shape=(2, 3), dtype=float32)

r_hdagextra=torch.tensor([-0.38711396,0.34880346])
python_hdagextra=h_extrapolated(thetas_python, targets, min, max)

print(f'h_dag_extra: {python_hdagextra}')
print(f'differnece r python : {r_hdagextra-python_hdagextra}')


h_dag_extra: tensor([-0.38710955,  0.34880140])
differnece r python : tensor([-4.41074371e-06,  2.05636024e-06])


### h_dag_dash_extrapolated

In [35]:

def h_dash_extrapolated(thetas: torch.Tensor, targets: torch.Tensor, k_min: float, k_max: float) -> torch.Tensor:
    """
    Extrapolated version of h_dag_dash for out-of-domain values.
    
    Args:
        t_targetsi: shape (n,)
        thetas: shape (n, b)
        k_min: float (not tracked by autograd)
        k_max: float (not tracked by autograd)
    
    Returns:
        Tensor: shape (n,)
    """
    # Constants
    L_START = 0.0001
    R_START = 1.0-L_START

    # Detach constants from graph
    L_tensor = torch.tensor(L_START, dtype=targets.dtype, device=targets.device)
    R_tensor = torch.tensor(R_START, dtype=targets.dtype, device=targets.device)

    # Scale input
    t_scaled = (targets - k_min) / (k_max - k_min)
    t_exp = t_scaled.unsqueeze(-1)  # shape (n, 1)

    # Left extrapolation: constant slope at L_START
    slope0 = h_dag_dash(L_tensor.expand_as(targets), thetas).unsqueeze(-1)  # (n, 1)
    mask0 = t_exp < L_tensor
    h_dash = torch.where(mask0, slope0, t_exp)  # placeholder init

    # Right extrapolation: constant slope at R_START
    slope1 = h_dag_dash(R_tensor.expand_as(targets), thetas).unsqueeze(-1)  # (n, 1)
    mask1 = t_exp > R_tensor
    h_dash = torch.where(mask1, slope1, h_dash)

    # In-domain interpolation
    mask_mid = (t_exp >= L_tensor) & (t_exp <= R_tensor)
    h_center = h_dag_dash(t_scaled, thetas).unsqueeze(-1)  # (n, 1)
    h_dash = torch.where(mask_mid, h_center, h_dash)

    return h_dash.squeeze(-1)  # shape (n,)


# h_dag_dash_extra(t_i[,cont_dims, drop=FALSE],
#                      theta[,cont_dims,1:len_theta,drop=FALSE],
#                      k_min[cont_dims],
#                      k_max[cont_dims])
# tf.Tensor(
# [[0.6818857  0.7085784  0.7162883 ]
#  [0.6908235  0.66555375 0.6818333 ]], shape=(2, 3), dtype=float32)

r_hdag_dash_extra=torch.tensor([0.6818857 ,0.6908235])
python_hdag_dash_extra=h_dash_extrapolated(thetas_python, targets, min, max)

print(f'h_dag_dash_extra: {python_hdag_dash_extra}')
print(f'differnece r python : {r_hdag_dash_extra-python_hdag_dash_extra}')

h_dag_dash_extra: tensor([0.68188578, 0.69082350])
differnece r python : tensor([-5.96046448e-08,  0.00000000e+00])


# Works to E-5 until here

## wrap all in loss fct

In [48]:
# skeleton for loss analog to R CODE  all the 

def contram_nll(outputs, targets, min_max):
    """
    Args:
        outputs: dict with keys 'int_out' and 'shift_out'
        targets: shape (n,)
        min_max: tuple of two floats or tensors (min, max)
    Returns:
        scalar NLL
    """
    # Ensure min and max are not part of graph
    min_val = torch.tensor(min_max[0], dtype=targets.dtype, device=targets.device)
    max_val = torch.tensor(min_max[1], dtype=targets.dtype, device=targets.device)

    thetas_tilde = outputs['int_out']  # shape (n, b)
    thetas = transform_intercepts_continous(thetas_tilde)

    Shifts = outputs['shift_out']  # shape (n,)
    

    # Compute h
    h_I = h_extrapolated(thetas, targets, min_val, max_val)  # shape (n,)
    h = h_I + torch.sum(Shifts)  # shape (n,)

    # Latent logistic density log-prob
    log_latent_density = -h - 2 * torch.nn.functional.softplus(-h)  # shape (n,)

    # Derivative term (log |h'| - log(scale))
    h_dash = h_dash_extrapolated(thetas, targets, min_val, max_val)  # shape (n,)
    log_hdash = torch.log(torch.abs(h_dash)) - torch.log(max_val - min_val)  # shape (n,)

    # Final NLL
    nll = -torch.mean(log_latent_density + log_hdash)

    return nll


## test los step by step

In [ ]:
#train$df_orig
# tf.Tensor(
# [[ 0.38709584 -0.18872575  3.9523134 ]
#  [ 0.7616431  -0.11664478 -1.4364368 ]], shape=(2, 3), dtype=float32)

In [54]:
int_out=theta_tilde_r
shifts_out=torch.tensor([[0,  0 ]])
outputs={'int_out':int_out,'shift_out':shifts_out}


targets=torch.tensor([ 0.38709584, 0.7616431 ])
min=0.4058232
max=0.74291575
min_max=torch.Tensor([min, max])


# contram_nll(outputs, targets, min_max)
min_val = torch.tensor(min_max[0], dtype=targets.dtype, device=targets.device)
max_val = torch.tensor(min_max[1], dtype=targets.dtype, device=targets.device)

Shifts = outputs['shift_out']

thetas_tilde = outputs['int_out'] 
thetas = transform_intercepts_continous(thetas_tilde)


/tmp/ipykernel_1320659/1406563847.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_val = torch.tensor(min_max[0], dtype=targets.dtype, device=targets.device)
/tmp/ipykernel_1320659/1406563847.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_val = torch.tensor(min_max[1], dtype=targets.dtype, device=targets.device)


In [55]:
r_hdagextra=torch.tensor([-0.38711396,0.34880346])
h_I=h_extrapolated(thetas_python, targets, min, max)

print(f'h_dag_extra: {h_I}')
print(f'differnece r python : {r_hdagextra-h_I}')

h_dag_extra: tensor([-0.38710955,  0.34880140])
differnece r python : tensor([-4.41074371e-06,  2.05636024e-06])


In [56]:
h = h_I + torch.sum(Shifts)
h   # same becasue shifts are 0

tensor([-0.38710955,  0.34880140])

In [57]:
# tf.Tensor(
# [[-1.423527  -1.4178157 -1.4313766]
#  [-1.4165573 -1.4279113 -1.411321 ]], shape=(2, 3), dtype=float32)


r_log_latent_density=torch.tensor([-1.423527 ,-1.4165573])


log_latent_density = -h - 2 * torch.nn.functional.softplus(-h)  # shape (n,)
log_latent_density


print(f'log_latent_density: {log_latent_density}')
print(f'differnece r python : {log_latent_density-r_log_latent_density}')

log_latent_density: tensor([-1.42352617, -1.41655695])
differnece r python : tensor([8.34465027e-07, 3.57627869e-07])


In [58]:
# tf.Tensor(
# [[0.68766224 0.63910955 0.6786249 ]
#  [0.7051456  0.7142741  0.70591867]], shape=(2, 3), dtype=float32)
r_h_dash=torch.tensor([0.68766224,0.7051456])
r_hdag_dash_extra=torch.tensor([0.6818857 ,0.6908235])



h_dash = h_dash_extrapolated(thetas, targets, min_val, max_val)  # shape (n,)

print(f'h_dash: {h_dash}')
print(f'differnece r python : {h_dash-r_h_dash}')

h_dash: tensor([0.68188578, 0.69082350])
differnece r python : tensor([-0.00577646, -0.01432210])


In [59]:
# tf.Tensor(
# [[ 0.7045046  2.390831  -1.9126254]
#  [ 0.7175269  2.3281896 -1.9619229]], shape=(2, 3), dtype=float32)
r_log_hdash=torch.tensor([0.7045046 ,0.7175269])


log_hdash = torch.log(torch.abs(h_dash)) - torch.log(max_val - min_val)  # shape (n,)
log_hdash

print(f'h_dash: {log_hdash}')
print(f'differnece r python : {r_log_hdash-log_hdash}')


h_dash: tensor([0.70450473, 0.71752691])
differnece r python : tensor([-1.19209290e-07,  0.00000000e+00])


In [ ]:
# NLL = NLL - tf$reduce_mean(log_latent_density + log_hdash)
